In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [22]:
df = pd.read_csv('FireWall_train.csv')

In [23]:
df.head(10)

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Action
0,63774,445,0,0,70,70,0,1,0,1,0,drop
1,64955,443,38956,443,2279,1129,1150,25,120,12,13,allow
2,52781,445,0,0,66,66,0,1,0,1,0,drop
3,64490,53,23595,53,280,118,162,2,30,1,1,allow
4,60927,445,0,0,70,70,0,1,0,1,0,drop
5,65117,445,0,0,66,66,0,1,0,1,0,drop
6,1594,44847,0,0,62,62,0,1,0,1,0,deny
7,53652,1900,0,0,139,139,0,1,0,1,0,deny
8,54963,53,23635,53,210,110,100,2,30,1,1,allow
9,50514,445,0,0,70,70,0,1,0,1,0,drop


In [24]:
df1=df.drop(['Source Port','Bytes','Bytes Sent','pkts_sent'],axis=1)

In [27]:
df1.Action.unique()

array([1, 0, 2, 3])

In [26]:
df1.loc[df['Action'] == 'allow', 'Action'] = 0
df1.loc[df['Action'] == 'drop', 'Action'] = 1
df1.loc[df['Action'] == 'deny', 'Action'] = 2
df1.loc[df['Action'] == 'reset-both', 'Action'] = 3
df1['Action'] = df1['Action'].astype(int)

In [17]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df1['Action'] = labelencoder.fit_transform(df1['Action'])

In [28]:
X=df1[['Destination Port','NAT Source Port','NAT Destination Port',
       'Bytes Received','Packets','Elapsed Time (sec)','pkts_received']]
y=df1.Action
y

0        1
1        0
2        1
3        0
4        1
        ..
49995    0
49996    2
49997    0
49998    0
49999    0
Name: Action, Length: 50000, dtype: int32

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [30]:
model = LogisticRegression(random_state=2).fit(X_train,y_train)

C:\Users\Sudip\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
y_predict = model.predict(X_test)
y_predict

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
income_acc = accuracy_score(y_test,y_predict)
income_f1_macro = f1_score(y_test,y_predict,average='macro')
income_confusion=confusion_matrix(y_test,y_predict)
total1=sum(sum(income_confusion))
income_sensitivity=income_confusion[0,0]/(income_confusion[0,0]+income_confusion[0,1])
income_specificity=income_confusion[1,1]/(income_confusion[1,0]+income_confusion[1,1])

print('Accuracy:',income_acc)
print('F1 Macro:',income_f1_macro)
print('Confusion Matrix:',income_confusion)
print('Sensitivity:',income_sensitivity)
print('Specificity:',income_specificity)
income_confusion[0]

Accuracy: 0.9835757575757575
F1 Macro: 0.7316385265661157
Confusion Matrix: [[9428   12    1    0]
 [   0 3258    0    0]
 [   5  241 3543    0]
 [   2    1    9    0]]
Sensitivity: 0.9987288135593221
Specificity: 1.0


array([9428,   12,    1,    0], dtype=int64)

In [35]:
df_test=pd.read_csv('FireWall_test.csv')

In [36]:
df_test

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,50626,443,22681,443,1604,987,617,16,93,9,7
1,51587,445,0,0,70,70,0,1,0,1,0
2,9337,443,53921,443,428,128,300,8,47,3,5
3,13554,44847,0,0,62,62,0,1,0,1,0
4,60985,443,19268,443,20423,4091,16332,33,122,15,18
...,...,...,...,...,...,...,...,...,...,...,...
15527,41693,443,28921,443,27069,3064,24005,44,178,19,25
15528,65451,53,3084,53,728,234,494,6,67,3,3
15529,54301,445,0,0,70,70,0,1,0,1,0
15530,55056,44512,0,0,62,62,0,1,0,1,0


In [37]:
df_test1=df_test.drop(['Source Port','Bytes','Bytes Sent','pkts_sent'],axis=1)

In [38]:
df_test1

,Destination Port,NAT Source Port,NAT Destination Port,Bytes Received,Packets,Elapsed Time (sec),pkts_received
0,443,22681,443,617,16,93,7
1,445,0,0,0,1,0,0
2,443,53921,443,300,8,47,5
3,44847,0,0,0,1,0,0
4,443,19268,443,16332,33,122,18
...,...,...,...,...,...,...,...
15527,443,28921,443,24005,44,178,25
15528,53,3084,53,494,6,67,3
15529,445,0,0,0,1,0,0
15530,44512,0,0,0,1,0,0


In [39]:
y_predict1 = model.predict(df_test1)
y_predict1

array([0, 1, 0, ..., 1, 2, 2])

In [49]:
output = pd.DataFrame(y_predict1).reset_index()
output.columns = ['Index','Action']
output

,Index,Action
0,0,0
1,1,1
2,2,0
3,3,2
4,4,0
...,...,...
15527,15527,0
15528,15528,0
15529,15529,1
15530,15530,2


In [50]:
output.to_csv('prediction.csv',header=True,index=False)

# Complete Code

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


df = pd.read_csv('FireWall_train.csv')

df1=df.drop(['Source Port','Bytes','Bytes Sent','pkts_sent'],axis=1)

df1.loc[df['Action'] == 'allow', 'Action'] = 0
df1.loc[df['Action'] == 'drop', 'Action'] = 1
df1.loc[df['Action'] == 'deny', 'Action'] = 2
df1.loc[df['Action'] == 'reset-both', 'Action'] = 3
df1['Action'] = df1['Action'].astype(int)

X=df1[['Destination Port','NAT Source Port','NAT Destination Port',
       'Bytes Received','Packets','Elapsed Time (sec)','pkts_received']]
y=df1.Action

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = LogisticRegression(random_state=2).fit(X_train,y_train)

y_predict = model.predict(X_test)

income_acc = accuracy_score(y_test,y_predict)
income_f1_macro = f1_score(y_test,y_predict,average='macro')
income_confusion=confusion_matrix(y_test,y_predict)
total1=sum(sum(income_confusion))
income_sensitivity=income_confusion[0,0]/(income_confusion[0,0]+income_confusion[0,1])
income_specificity=income_confusion[1,1]/(income_confusion[1,0]+income_confusion[1,1])

print('Accuracy:',income_acc)
print('F1 Macro:',income_f1_macro)
print('Confusion Matrix:',income_confusion)
print('Sensitivity:',income_sensitivity)
print('Specificity:',income_specificity)
income_confusion[0]

df_test=pd.read_csv('FireWall_test.csv')

df_test1=df_test.drop(['Source Port','Bytes','Bytes Sent','pkts_sent'],axis=1)

y_predict1 = model.predict(df_test1)

output = pd.DataFrame(y_predict1).reset_index()
output.columns = ['Index','Action']

output.to_csv('prediction.csv',header=True,index=False)